## [python] 연관규칙분석(ASSOCIATION RULE ANALYSIS) - 장바구니분석
### 개요
추천 시스템이란, '특정 시점'에 '특정 고객'이 좋아할 만한 리스트를 추려내는 것이다. 추천 시스템을 구현하는 데는 다음과 같은 세 가지 데이터를 활용한다 - 1) 고객의 정보, 2) 가지고 있는 전체 리스트(아이템), 3) 시스템 외부 정보가 그것이다.

많은 추천 시스템의 알고리즘이 있지만, 인프라 추천을 위해 다음과 같은 장점을 가진 Association Rule Analysis(연관규칙분석)을 활용하였다.  
1) Association Rule Analysis은 cold-start problem을 풀기에 좋다. 즉, non-personalized 알고리즘으로 고객에 대한 정보가 없을 경우 좋은 선택이다.  
2) Association Rule Analysis는 연산량이 적음에도 불구하고 비교적 좋은 성능을 낸다.  

따라서, 인프라 딜 하나 하나를 발생한 구매라고 생각하고, 각종 지표(국가, 인프라 금액)를 구매한 아이템이라고 가정하여 연관규칙분석을 실시하였다.

### Reference:
https://hezzong.tistory.com/entry/python-%EC%97%B0%EA%B4%80%EA%B7%9C%EC%B9%99%EB%B6%84%EC%84%9DA-Priori-Algorithm
https://yamalab.tistory.com/67

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
# The duplicated ProjectIDs were deleted before loading the excel file
data = pd.read_excel('PPIDB_2019_final (BF) final.xlsx', sheet_name='Base')
data.head()

,Region,Country,IncomeGroup,IDAStatus,Financialclosureyear,FinancialclosureMonth,ProjectID,Projectname,RelatedNames,TypeofPPI,...,incomelevel,incomelevelname,lendingtype,lendingtypename,GDP,population,CPI2019,realinvestment,realphysicalassets,realfeestogovernment
0,Latin America and the Caribbean,Mexico,Upper middle income,Non-IDA,1990,November,991,Telefonos de Mexico (Telmex),Telmex,Divestiture,...,UMC,Upper middle income,IBD,IBRD,2.612536e+11,83943132.0,1.921853,4001.298828,0.000000,4001.298828
1,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,1990,September,1330,"Intercity Roads - Corridor 7, 8 & 9",Servicios Viales S.A.,Brownfield,...,HIC,High income,IBD,IBRD,1.413524e+11,32618651.0,1.921853,340.168060,340.168060,0.000000
2,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,1990,September,1329,Intercity Roads - Corridor 6,Concessionario Vial Corridor Route 188 UTE - C...,Brownfield,...,HIC,High income,IBD,IBRD,1.413524e+11,32618651.0,1.921853,82.639694,82.639694,0.000000
3,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,1990,September,1331,Intercity Roads - Corridor 10,Covicentro S.A.,Brownfield,...,HIC,High income,IBD,IBRD,1.413524e+11,32618651.0,1.921853,186.419785,186.419785,0.000000
4,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,1990,September,3179,Camino del Atlantico SA CV,"Vial interconection system of Atlantic region,...",Brownfield,...,HIC,High income,IBD,IBRD,1.413524e+11,32618651.0,1.921853,NaN,0.000000,0.000000


In [3]:
# check columns
data.columns

Index(['Region', 'Country', 'IncomeGroup', 'IDAStatus', 'Financialclosureyear',
       'FinancialclosureMonth', 'ProjectID', 'Projectname', 'RelatedNames',
       'TypeofPPI', 'SubtypeofPPI', 'Projectstatus', 'dateStatusUpdated',
       'Primarysector', 'SubSector', 'Segment', 'Location', 'ContractPeriod',
       'GovtGrantingContract', 'DirectGovtSupport', 'DirectGovtSupportValue',
       'InDirectGovtSupport', 'InDirectGovtSupportValue', 'InvestmentYear',
       'PercentPrivate', 'FeesToGovernment', 'PhysicalAssets',
       'TotalInvestment', 'CapacityType', 'Capacity', 'Technology',
       'RelatedProjects', 'BidCriteria', 'AwardMethod', 'NumberOfBids',
       'Sponsors', 'MainRevenueSource', 'OtherRevenueSource',
       'MultiLateralSupport', 'BiLateralSupport', 'Description', 'FundingYear',
       'TotalDebtFunding', 'DebtEquityGrantRatio', 'TotalEquity',
       'ProjectBanks', 'CommercialDebt', 'MultilateralDebt', 'BilateralDebt',
       'InstitutionalDebt', 'PublicDebt', 'Intern

In [4]:
# put out some columns that are not meaningful
# 사람이 판단하기는 어렵지만, 애초에 의미가 없는 것들 (Unique Project ID 등) 및 연속 데이터 삭제
# ProjectID'(This would be the index)
col = ['Region', 'Country', 'IncomeGroup', 'IDAStatus', 'TypeofPPI', 'SubtypeofPPI', 'Projectstatus',
       'Primarysector', 'SubSector', 'Segment']

In [5]:
data.set_index('ProjectID', inplace=True)
data_cut = data[col]
data_cut.dropna()
data_cut.head()

,Region,Country,IncomeGroup,IDAStatus,TypeofPPI,SubtypeofPPI,Projectstatus,Primarysector,SubSector,Segment
ProjectID,,,,,,,,,,
991,Latin America and the Caribbean,Mexico,Upper middle income,Non-IDA,Divestiture,Full,Active,ICT,ICT backbone,Other
1330,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,Brownfield,"Rehabilitate, operate, and transfer",Concluded,Transport,Roads,Highway
1329,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,Brownfield,"Rehabilitate, operate, and transfer",Concluded,Transport,Roads,Highway
1331,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,Brownfield,"Rehabilitate, operate, and transfer",Concluded,Transport,Roads,Highway
3179,Latin America and the Caribbean,Argentina,Upper middle income,Non-IDA,Brownfield,"Rehabilitate, operate, and transfer",Concluded,Transport,Roads,Highway


In [6]:
dataset = []
for i in range(0, len(data_cut)):
    ele = data_cut.iloc[i].tolist()
    dataset.append(ele)

In [7]:
dataset

[['Latin America and the Caribbean',
  'Mexico',
  'Upper middle income',
  'Non-IDA',
  'Divestiture',
  'Full',
  'Active',
  'ICT',
  'ICT backbone',
  'Other'],
 ['Latin America and the Caribbean',
  'Argentina',
  'Upper middle income',
  'Non-IDA',
  'Brownfield',
  'Rehabilitate, operate, and transfer',
  'Concluded',
  'Transport',
  'Roads',
  'Highway'],
 ['Latin America and the Caribbean',
  'Argentina',
  'Upper middle income',
  'Non-IDA',
  'Brownfield',
  'Rehabilitate, operate, and transfer',
  'Concluded',
  'Transport',
  'Roads',
  'Highway'],
 ['Latin America and the Caribbean',
  'Argentina',
  'Upper middle income',
  'Non-IDA',
  'Brownfield',
  'Rehabilitate, operate, and transfer',
  'Concluded',
  'Transport',
  'Roads',
  'Highway'],
 ['Latin America and the Caribbean',
  'Argentina',
  'Upper middle income',
  'Non-IDA',
  'Brownfield',
  'Rehabilitate, operate, and transfer',
  'Concluded',
  'Transport',
  'Roads',
  'Highway'],
 ['Latin America and the Ca

In [8]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

"""
주어진 코드에서 fit 함수를 통해 dataset은 고유한 라벨을 갖게 되고,
transform함수를 통해서 파이썬 리스트를 one-hot 인코딩 된 numPy 배열로 변환합니다.
one-hot 인코딩에 대한 자세한 설명을 원하시면 다음 글을 참고해 주시기 바랍니다.
(https://teddylee777.github.io/machine-learning/python-numpy%EB%A1%9C-one-hot-encoding-%EC%89%BD%EA%B2%8C%ED%95%98%EA%B8%B0)
"""

'\n주어진 코드에서 fit 함수를 통해 dataset은 고유한 라벨을 갖게 되고,\ntransform함수를 통해서 파이썬 리스트를 one-hot 인코딩 된 numPy 배열로 변환합니다.\none-hot 인코딩에 대한 자세한 설명을 원하시면 다음 글을 참고해 주시기 바랍니다.\n(https://teddylee777.github.io/machine-learning/python-numpy%EB%A1%9C-one-hot-encoding-%EC%89%BD%EA%B2%8C%ED%95%98%EA%B8%B0)\n'

In [9]:
# 지지도가 0.5 이상인 항목들
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

In [10]:
frequent_itemsets

,support,itemsets
0,0.931726,(Active)
1,0.509605,(Energy)
2,0.643338,(Greenfield project)
3,0.899914,(Non-IDA)
4,0.687018,(Upper middle income)
5,0.618622,"(Greenfield project, Active)"
6,0.843999,"(Non-IDA, Active)"
7,0.643950,"(Upper middle income, Active)"
8,0.573718,"(Greenfield project, Non-IDA)"
9,0.684204,"(Upper middle income, Non-IDA)"


In [11]:
# antecedents: 조건절 / consequents: 결과절
association_rules(frequent_itemsets, metric="lift", min_threshold=1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Greenfield project),(Active),0.643338,0.931726,0.618622,0.961582,1.032044,0.019207,1.777141
1,(Active),(Greenfield project),0.931726,0.643338,0.618622,0.663953,1.032044,0.019207,1.061345
2,(Non-IDA),(Active),0.899914,0.931726,0.843999,0.937865,1.006589,0.005525,1.098801
3,(Active),(Non-IDA),0.931726,0.899914,0.843999,0.905844,1.006589,0.005525,1.062974
4,(Upper middle income),(Active),0.687018,0.931726,0.643950,0.937311,1.005994,0.003837,1.089080
5,(Active),(Upper middle income),0.931726,0.687018,0.643950,0.691136,1.005994,0.003837,1.013332
6,(Upper middle income),(Non-IDA),0.687018,0.899914,0.684204,0.995904,1.106665,0.065947,24.433953
7,(Non-IDA),(Upper middle income),0.899914,0.687018,0.684204,0.760299,1.106665,0.065947,1.305718
8,"(Greenfield project, Non-IDA)",(Active),0.573718,0.931726,0.553530,0.964811,1.035509,0.018981,1.940211
9,"(Non-IDA, Active)",(Greenfield project),0.843999,0.643338,0.553530,0.655842,1.019437,0.010554,1.036333


## 결론
#### 1. "조건절"을 좋아하는 사람은 "결과절"도 좋아하였다: 조건절에 맞는 프로젝트를 보는 경우, 결과절에 해당하는 프로젝트를 띄워줄 수 있겠음.
#### 2. 국가와 다른 항목들의 연관관계는 위에서는 나타나지 않으나, 한 column당 element 종류가 확률에 영향을 주기 때문에 (국가의 수는 region의 수보다 많음) / 1에서 조건절과 결과절을 이용하여 See Also를 띄워줄 때 국가나 Region을 맞추어 띄워주는 것이 좋겠음.
#### 3. 연속변수의 경우 범주형으로 변환하여 분석해보면 다른 재미있는 결과가 나올 것이라고 생각함